# Setup

In [210]:
import pandas as pd
import numpy as np
import scipy

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

***
# Завдання першого рівня
Виконати всі завдання, використовуючи як numpy array, так і dataframe, проаналізувати часові витрати на виконання процедур (профілювання часу виконання), зробити висновки щодо ситуацій, в яких має сенс віддати перевагу тій чи іншій структурі даних. Висновки оформити звітом із зазначеним часом виконання та оцінкою по 5-бальній шкалі зручності виконання операцій відбору).

## Reading and Dropping

In [211]:
dictionary_for_needed_type = {
    'Global_active_power': 'float64',
    'Global_reactive_power': 'float64',
    'Voltage': 'float64',
    'Global_intensity': 'float64',
    'Sub_metering_1': 'float64',
    'Sub_metering_2': 'float64',
    'Sub_metering_3': 'float64',
                             }

In [212]:
df = pd.read_csv("household_power_consumption.txt", sep=";", low_memory=False)
nar = df.values

In [213]:
df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)

nar = np.delete(nar, np.where(nar == '?'), 0)

In [214]:
df = df.astype(dictionary_for_needed_type)
ar = np.concatenate((nar[:, [0, 1]], nar[:, [2, 3, 4, 5]].astype(float), nar[:, [6, 7, 8]].astype(float).astype(int)), axis=1)

## 1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [215]:
def get_df_first(df):
    df_first = df[df["Global_active_power"]>5][["Date", "Time", 'Global_active_power', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
    return df_first
    
    
def get_ar_first(ar):
    ar_first = ar[ar[..., 2]>5]
    return ar_first

%timeit -r 2 -n 100 get_df_first(df)
%timeit -r 2 -n 100 get_ar_first(ar)

df_first = get_df_first(df)
ar_first = get_ar_first(ar)

5.9 ms ± 549 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
49 ms ± 29.8 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)


In [216]:
df_first

,Date,Time,Global_active_power,Sub_metering_1,Sub_metering_2,Sub_metering_3
1,16/12/2006,17:25:00,5.360,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.0,1.0,17.0
11,16/12/2006,17:35:00,5.412,0.0,1.0,17.0
12,16/12/2006,17:36:00,5.224,0.0,1.0,16.0
...,...,...,...,...,...,...
2069356,22/11/2010,18:40:00,5.408,48.0,0.0,0.0
2069357,22/11/2010,18:41:00,5.528,53.0,0.0,0.0
2071586,24/11/2010,07:50:00,5.172,0.0,38.0,17.0
2071587,24/11/2010,07:51:00,5.750,0.0,39.0,17.0


In [217]:
pd.DataFrame.from_records(ar_first)[[0, 1, 2, 6, 7, 8]]

,0,1,2,6,7,8
0,16/12/2006,17:25:00,5.360,0,1,16
1,16/12/2006,17:35:00,5.412,0,1,17
2,16/12/2006,17:36:00,5.224,0,1,16
3,16/12/2006,17:37:00,5.268,0,2,17
4,16/12/2006,17:44:00,5.894,0,0,16
...,...,...,...,...,...,...
17540,22/11/2010,18:40:00,5.408,48,0,0
17541,22/11/2010,18:41:00,5.528,53,0,0
17542,24/11/2010,07:50:00,5.172,0,38,17
17543,24/11/2010,07:51:00,5.750,0,39,17


## 2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [218]:
def get_df_second(df):
    df_second = df[df["Voltage"]>235][["Date", "Time", 'Voltage', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
    return df_second
    
    
def get_ar_second(ar):
    ar_second = ar[ar[..., 4]>235]
    return ar_second


%timeit -r 2 -n 100 get_df_second(df)
%timeit -r 2 -n 100 get_ar_second(ar)

df_second = get_df_second(df)
ar_second = get_ar_second(ar)

156 ms ± 1.75 ms per loop (mean ± std. dev. of 2 runs, 100 loops each)
187 ms ± 1.52 ms per loop (mean ± std. dev. of 2 runs, 100 loops each)


In [219]:
df_second

,Date,Time,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3
4,16/12/2006,17:28:00,235.68,0.0,1.0,17.0
5,16/12/2006,17:29:00,235.02,0.0,2.0,17.0
6,16/12/2006,17:30:00,235.09,0.0,1.0,17.0
7,16/12/2006,17:31:00,235.22,0.0,1.0,17.0
14,16/12/2006,17:38:00,235.24,0.0,1.0,17.0
...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,240.43,0.0,0.0,0.0
2075255,26/11/2010,20:59:00,240.00,0.0,0.0,0.0
2075256,26/11/2010,21:00:00,239.82,0.0,0.0,0.0
2075257,26/11/2010,21:01:00,239.70,0.0,0.0,0.0


In [220]:
pd.DataFrame.from_records(ar_second)[[0, 1, 4, 6, 7, 8]]

,0,1,4,6,7,8
0,16/12/2006,17:38:00,235.24,0,1,17
1,16/12/2006,17:39:00,237.14,0,0,17
2,16/12/2006,17:40:00,236.73,0,0,17
3,16/12/2006,17:41:00,237.06,0,0,17
4,16/12/2006,17:42:00,237.13,0,0,18
...,...,...,...,...,...,...
1952482,26/11/2010,20:58:00,240.43,0,0,0
1952483,26/11/2010,20:59:00,240.00,0,0,0
1952484,26/11/2010,21:00:00,239.82,0,0,0
1952485,26/11/2010,21:01:00,239.70,0,0,0


## 3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [ ]:
def get_df_third(df):
    df_third = df[(df["Global_intensity"] >= 19) & (df["Global_intensity"] <= 20) & (df["Sub_metering_2"] > df["Sub_metering_3"])][["Date", "Time", 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
    return df_third
    
    
def get_ar_third(ar):
    ar_third = ar[(ar[:, 5]>=19) & (ar[:, 5]<=20) & (ar[:, 7]>ar[:, 8])]
    return ar_third


%timeit -r 2 -n 100 get_df_third(df)
%timeit -r 2 -n 100 get_ar_third(ar)

df_third = get_df_third(df)
ar_third = get_ar_third(ar)

9.57 ms ± 122 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)


In [ ]:
df_third

In [ ]:
ar_third

## 4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії, а також

In [ ]:
random_df = df.sample(500000)
random_ar = random_df.values

    
    
def get_df_fourth(df):
    random_df['Average_of_3'] = (random_df['Sub_metering_1']+random_df['Sub_metering_2']+random_df['Sub_metering_3'])/3
    df_fourth = random_df[["Date", "Time", 'Average_of_3', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
    return df_fourth
    
    
def get_ar_fourth(ar):
    a = np.average(random_ar[..., [6, 7, 8]], axis=1)
    ar_fourth = np.column_stack((random_ar[:, [0, 1, 6, 7, 8]], a.reshape(a.size, 1)))
    return ar_fourth


%timeit -r 2 -n 100 get_df_fourth(df)
%timeit -r 2 -n 100 get_ar_fourth(ar)

df_fourth = get_df_fourth(df)
ar_fourth = get_ar_fourth(ar)

In [ ]:
df_fourth

In [ ]:
ar_fourth

## 5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [ ]:
def get_fifth_df(df):
    fifth_df = df[(df['Time']>= "18:30:00") & (df['Global_active_power']>6) & (df["Sub_metering_2"]>df["Sub_metering_1"]) & (df["Sub_metering_2"]>df["Sub_metering_3"])]
    # fifth_df
    order = []
    for i in range(0, len(fifth_df)//2, 3):
        order.append(fifth_df.iloc[i])
    for i in range(len(fifth_df)//2, len(fifth_df), 4):
        order.append(fifth_df.iloc[i])

    df_fifth = pd.DataFrame.from_records(order)
    # print(df_fifth)
    return df_fifth
    
    
def get_fifth_ar(ar):
    ar_fifth = ar[(ar[..., 1]>="18:30:00") & (ar[..., 2]>6) & (ar[..., 7]>ar[..., 8])& (ar[..., 7]>ar[..., 6])]
    order = []
    for i in range(0, len(ar_fifth)//2, 3):
        order.append(ar_fifth[i])
    for i in range(len(ar_fifth)//2, len(ar_fifth), 4):
        order.append(ar_fifth[i])
    # end_time = np.datetime64('now', 'ms')
    fifth_ar = pd.DataFrame.from_records(order).values
    return fifth_ar


%timeit -r 2 -n 10 get_fifth_df(df)
%timeit -r 2 -n 10 get_fifth_ar(ar)

fifth_df = get_fifth_df(df)
fifth_ar = get_fifth_ar(ar)

In [ ]:
fifth_df

In [ ]:
fifth_ar

***
# *Завдання другого рівня*

Виконати всі завдання, використовуючи як numpy array, так і dataframe

Датасет має відповідати таким вимогам:
- Data Set Characteristics: Multivariate
- Attribute Characteristics: Categorical, Integer, Real
- Number of Attributes: at least 2 integers/real
- Missing Values? YES!!!!!

In [ ]:
df = pd.read_csv("pokemon.csv")

In [ ]:
df.isnull().sum()

## 1. Поборотися із зниклими даними.

In [ ]:
df[['height_m', 'weight_kg']] = df[['height_m', 'weight_kg']].bfill()
# df = df.fillna(method='ffill')

In [ ]:
df.isnull().sum()

## 2. Пронормувати вибраний датасет або стандартизувати його.

In [ ]:
df_num = df[['attack', 'defense', 'hp', "speed", "sp_attack", "sp_defense"]]

normalized_df_num = (df_num-df_num.min())/(df_num.max()-df_num.min())
#normalized_df_num.max()

copy_df = df.copy()
copy_df[['attack', 'defense', 'hp', "speed", "sp_attack", "sp_defense"]] = normalized_df_num

normilized_df = copy_df
# normilized_df
#standatrized_df_num = (df_num - df_num.mean()) / df_num.std()
#print(standatrized_df_num.mean().apply(lambda x: "{:.6f}".format(x)))
#print(standatrized_df_num.std())
#print(standatrized_df_num.var())
#standatrized_df_num

## 3. Збудувати гістограму по одному із атрибутів, що буде показувати на кількість елементів, що знаходяться у 10 діапазонах, які ви задасте.

In [ ]:
sns.set_style("white")
sns.histplot(normilized_df[["speed"]], bins=10)

## 4. Збудувати графік залежності одного integer/real атрибута від іншого.

In [ ]:
sns.set_style("whitegrid")
sns.lineplot(normilized_df[["against_flying", 'against_water']], y='against_flying', x='against_water') #, hue='is_legendary')

## 5. Підрахувати коефіцієнт Пірсона та Спірсона для двох integer/real атрибутів.

In [ ]:
pear = scipy.stats.pearsonr(normilized_df["attack"], normilized_df['sp_attack'])
spear = scipy.stats.spearmanr(normilized_df["attack"], normilized_df['sp_attack'])

print(f"Для Атаки та СпецАтаки: \nкоефіцієнт Пірсона: {pear}, \nкоефіцієнт Спірсона: {spear}.")

## 6. Провести One Hot Encoding категоріального string атрибуту.

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [ ]:
a = ohe.fit(normilized_df['type1'].values.reshape(-1, 1))

In [ ]:
ohe.categories_

## 7. Провести візуалізацію багатовимірних даних, використовуючи приклади.

In [ ]:
sns.pairplot(normilized_df[['attack', 'defense', 'hp', "speed", "sp_attack", "sp_defense", 'is_legendary']], hue="is_legendary", kind="reg")

***
# *Додаткове завдання*:  

## 8. Поділити випадковим чином датасет на дві рівні частини. Навчити 3 регресійні моделі на основі не менше одного атрибуту відновлювати інший. Навчання має відбуватися на основі [першого датасету](https://scikit-learn.org/stable/modules/linear_model.html), візуалізувати моделі та на основі [середньої квадратичної помилки](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) вибрати найкращу.

In [ ]:
train_set, test_set = train_test_split(normilized_df, test_size=0.5, random_state=42)

In [ ]:
test_set = test_set[:400]

In [ ]:
train_set_a = np.atleast_2d(train_set['against_flying'])#.values.reshape(-1, 1)
train_set_sa = np.atleast_2d(train_set['against_water'])#.values.reshape(-1, 1)

dfplot_a = normilized_df['against_flying']#.values.reshape(-1, 1)
dfplot_sa = normilized_df['against_water']#.values.reshape(-1, 1)

test_set_a = np.atleast_2d(test_set['against_flying'])#.values.reshape(-1, 1)
test_set_sa = np.atleast_2d(test_set['against_water'])#.values.reshape(-1, 1)

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(train_set_a, train_set_sa)

In [ ]:
# Навчання регресії опорних векторів
svr_reg = SVR(kernel='linear')
svr_reg.fit(train_set['against_flying'].values.reshape(-1, 1), train_set['against_water'].values)

In [ ]:
# Навчання регресії дерева рішень
tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_set_a, train_set_sa)

In [ ]:
# Візуалізація результатів
plt.figure(figsize=(12, 8))

# Лінійна регресія
plt.subplot(1, 3, 1)
plt.scatter(dfplot_a, dfplot_sa, color='blue')
plt.scatter(test_set_sa, linear_reg.predict(test_set_a), color='red')
plt.title('Лінійна регресія')

# Регресія опорних векторів
plt.subplot(1, 3, 2)
plt.scatter(dfplot_a, dfplot_sa, color='blue')
plt.scatter(test_set_sa.reshape(-1, 1), svr_reg.predict(test_set_a.reshape(-1, 1)), color='red')
plt.title('Регресія опорних векторів')

# Регресія дерева рішень
plt.subplot(1, 3, 3)
plt.scatter(dfplot_a, dfplot_sa, color='blue')
plt.scatter(test_set_sa, tree_reg.predict(test_set_a), color='red')
plt.title('Регресія дерева рішень')

plt.show()

In [ ]:
# Оцінка моделей за середньою квадратичною помилкою
mse_linear = mean_squared_error(test_set_sa, linear_reg.predict(test_set_a))
mse_svr = mean_squared_error(test_set_sa.reshape(-1, 1), svr_reg.predict(test_set_a.reshape(-1, 1)))
mse_tree = mean_squared_error(test_set_sa, tree_reg.predict(test_set_a))

print(f'MSE Лінійна регресія: {mse_linear}')
print(f'MSE Регресія опорних векторів: {mse_svr}')
print(f'MSE Регресія дерева рішень: {mse_tree}')